In [1]:
import sys
sys.path.append(r'C:\Calc\OpenIO_Data')
import openIO
import pandas as pd
import numpy as np

C:\Calc\OpenIO-Canada_ETS\openIO.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


To get the data from openIO-Canada, you simply need to enter 3 arguments:
- the path to the STATCAN economic data (Detailed level) for the year you chose (available here: https://www150.statcan.gc.ca/n1/en/catalogue/15-602-X). **Make sure to download the Excel versions for the code to work properly.**
- the path to an IOT pxp folder of Exiobase. Exiobase data can be found here: https://doi.org/10.5281/zenodo.5589597
- a boolean whether you wish to endogenize capitals or not. The default option is that they are endogenized. For more information on endogenization of capitals, read these articles: (https://doi.org/10.1021/acs.est.8b02791)
(https://doi.org/10.1111/jiec.12931). **Note that endogenization introduces a lot of uncertainties, but that not endogenizing capitals will result in emission factors not including the impact of infrastructure and in an underestimation of total emissions at provincial or national level.**

OpenIO-Canada takes a while to run (~45 minutes) on a good machine.

The command `tables = openIO.IOTables ... ` takes 20-50 minutes to run.

In [2]:
tables = openIO.IOTables(folder_path=r'C:\Calc\OpenIO_Data\StatCan\2021\Detail level',
                         exiobase_folder='C:\Calc\OpenIO_Data\Exiobase\monetary\exiobase_3.8.2\IOT_2021_pxp',
                         endogenizing_capitals=False)

2025-09-14 19:11:27,333 - openIO-Canada - INFO - Reading all the Excel files...
2025-09-14 19:12:01,039 - openIO-Canada - INFO - Formatting the Supply and Use tables...
2025-09-14 19:13:58,196 - openIO-Canada - INFO - Modifying names of duplicated sectors...
2025-09-14 19:13:59,001 - openIO-Canada - INFO - Organizing final demand sectors...
2025-09-14 19:14:00,999 - openIO-Canada - INFO - Removing IOIC codes from index...
2025-09-14 19:14:01,246 - openIO-Canada - INFO - Balancing inter-provincial trade...
2025-09-14 19:16:52,646 - openIO-Canada - INFO - Pre-treatment of international trade data...
2025-09-14 19:17:48,315 - openIO-Canada - INFO - Linking international trade data to openIO-Canada...
2025-09-14 19:19:56,162 - openIO-Canada - INFO - Building the symmetric tables...
2025-09-14 19:21:04,212 - openIO-Canada - INFO - Linking openIO-Canada to Exiobase...
2025-09-14 19:36:37,222 - openIO-Canada - INFO - Extracting and formatting environmental data from the NPRI file...
2025-09-1

Run the calc() function which will calculate the life cycle impacts of each sector.
The command `table.calc()` takes < 5 minutes.

In [3]:
tables.calc()

To avoid having to re-run the code everytime you want to use openIO-Canada, you can save the whole object in a pickle file.  Saving can take ~ 12 minutes, and creates a 2GB file.

In [5]:
import gzip
import pickle

with gzip.open(r'C:\Calc\OpenIO_Data\Output\openIO_2021_endogenize_False.gz.pickle', 'wb') as file:
    pickle.dump(tables, file)

Next cell: testing to save specific dataframes to excel for easier interrogation....

Note that this will not work for the large (16k x 16k matrices).  Too slow, and the file is too large to be useable.  Even saving a single matrix to excel can take 10 minutes.

ChatGPT suggests Parquet format.

Very compact, fast to read/write.

Readable in:
- Python (pandas, polars)
- R (arrow, data.table)
- Power BI / Tableau
- Excel (via Power Query → “Get Data” → “From Parquet”)

In [8]:
import os
import pandas as pd

# Example: your container variable
container = tables  # replace with your actual variable name

# Base directory you supply
base_dir = r'C:\Calc\OpenIO_Data\Output'

# Create a folder with the variable's name inside base_dir
folder_name = os.path.join(base_dir, "tables")   # "tables" is the variable name
os.makedirs(folder_name, exist_ok=True)

# Optional whitelist of DataFrame names to save
# Example: whitelist = ['A_exio', 'C',  'C_exio']
whitelist = ['C_exio']   # set to [] or None to save all

# Loop through attributes in the container
for attr_name in dir(container):
    if attr_name.startswith("_"):
        continue

    # If whitelist is provided, skip others
    if whitelist and attr_name not in whitelist:
        continue

    obj = getattr(container, attr_name)

    if isinstance(obj, pd.DataFrame):
        file_name = f"{attr_name}.xlsx"
        file_path = os.path.join(folder_name, file_name)

        # Status update
        print(f"Now saving dataframe {attr_name} as {file_name}...")

        # Save to Excel (includes index + column names)
        # Note there is no check for dataframe size; excel max sizes are 1.04e6 rows and 16,384 columns.  CanadaIO sizes top out at 16,196.
        obj.to_excel(file_path)

        print(f"\tFinished saving {attr_name}")


Now saving dataframe C_exio as C_exio.xlsx...
	Finished saving C_exio
